# 記事をElasticSearchに登録するnotebook

In [1]:
from elasticsearch import Elasticsearch
import os
from tqdm import tqdm

In [2]:
es = Elasticsearch("http://localhost:9200")

es.info().body

{'name': '9ebe0ce9d1d3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ZQ_xL79zTwas0WHCzaVckg',
 'version': {'number': '8.9.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d',
  'build_date': '2023-07-19T14:43:58.555259655Z',
  'build_snapshot': False,
  'lucene_version': '9.7.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [4]:
# インデックスの設定とマッピングを定義
index_name = "news_articles"
settings = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "kuromoji_tokenizer": {
                    "type": "kuromoji_tokenizer",
                    "mode": "search"
                }
            },
            "analyzer": {
                "my_ja_analyzer": {
                    "type": "custom",
                    "tokenizer": "kuromoji_tokenizer",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_part_of_speech",
                        "cjk_width",
                        "ja_stop",
                        "kuromoji_stemmer",
                        "lowercase"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "url": {
                "type": "text",
            },
            "title": {
                "type": "text",
                "analyzer": "my_ja_analyzer"
            },
            "content": {
                "type": "text",
                "analyzer": "my_ja_analyzer"
            },
            "published_at": {
                "type": "date",
            },
            "category": {
                "type": "keyword"
            }
        }
    }
}

# インデックスの作成
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=settings)
    print(f"インデックス '{index_name}' が作成されました。")
else:
    print(f"インデックス '{index_name}' は既に存在しています。")


インデックス 'news_articles' が作成されました。


In [5]:
# テキストファイルを処理してElasticsearchに登録する関数
def index_article(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    url = lines[0].strip()
    published_at = lines[1].strip()
    title = lines[2].strip()
    content = "".join(lines[3:]).strip()
    category = file_path.split("/")[-1].split("-")[0]


    doc = {
        "url": url,
        "published_at": published_at,
        "title": title,
        "content": content,
        "category": category
    }

    res = es.index(index=index_name, document=doc)
    print(f"ファイル'{file_path}'がインデックス化されました。ID: {res['_id']}")

In [6]:
directory_path = "/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data"
for filename in tqdm(os.listdir(directory_path)):
    file_path = os.path.join(directory_path, filename)
    index_article(file_path)


 15%|█▍        | 8/54 [00:00<00:01, 42.33it/s]

ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/kaden-channel-6912096.txt'がインデックス化されました。ID: vAStspIBHiy3rAkzNhp8
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/dokujo-tsushin-6903790.txt'がインデックス化されました。ID: vQStspIBHiy3rAkzNhq-
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/livedoor-homme-6828491.txt'がインデックス化されました。ID: vgStspIBHiy3rAkzNhrX
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/dokujo-tsushin-6908189.txt'がインデックス化されました。ID: vwStspIBHiy3rAkzNhr1
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/smax-6918624.txt'がインデックス化されました。ID: wAStspIBHiy3rAkzNxoG
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/dokujo-tsushin-6907725.txt'がインデックス化されました。ID: wQStspIBHiy3rAkzNxoT
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/movie-enter-6897614.txt'がインデックス化されました。ID: wgStspIBHiy3rAkzNxok
ファイル'/Users/pci/Desk

 39%|███▉      | 21/54 [00:00<00:00, 52.88it/s]

ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/it_life-hack-6917377.txt'がインデックス化されました。ID: yQStspIBHiy3rAkzNxqA
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/sports-watch-6911610.txt'がインデックス化されました。ID: ygStspIBHiy3rAkzNxqR
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/livedoor-homme-6783137.txt'がインデックス化されました。ID: ywStspIBHiy3rAkzNxqo
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/movie-enter-6900874.txt'がインデックス化されました。ID: zAStspIBHiy3rAkzNxrC
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/livedoor-homme-6754532.txt'がインデックス化されました。ID: zQStspIBHiy3rAkzNxrT
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/livedoor-homme-6767880.txt'がインデックス化されました。ID: zgStspIBHiy3rAkzNxrv
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/peachy-6907988.txt'がインデックス化されました。ID: zwStspIBHiy3rAkzNxr9
ファイル'/Users/pci/Deskt

 65%|██████▍   | 35/54 [00:00<00:00, 58.45it/s]

ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/topic-news-6903190.txt'がインデックス化されました。ID: 1AStspIBHiy3rAkzOBpG
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/sports-watch-6917848.txt'がインデックス化されました。ID: 1QStspIBHiy3rAkzOBpy
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/movie-enter-6909318.txt'がインデックス化されました。ID: 1gStspIBHiy3rAkzOBqD
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/dokujo-tsushin-6907399.txt'がインデックス化されました。ID: 1wStspIBHiy3rAkzOBqb
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/dokujo-tsushin-6915005.txt'がインデックス化されました。ID: 2AStspIBHiy3rAkzOBqu
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/smax-6917555.txt'がインデックス化されました。ID: 2QStspIBHiy3rAkzOBq-
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/it_life-hack-6918825.txt'がインデックス化されました。ID: 2gStspIBHiy3rAkzOBrS
ファイル'/Users/pci/Desktop/for

100%|██████████| 54/54 [00:00<00:00, 60.53it/s]

ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/smax-6917246.txt'がインデックス化されました。ID: 4wStspIBHiy3rAkzORoz
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/it_life-hack-6917568.txt'がインデックス化されました。ID: 5AStspIBHiy3rAkzORpC
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/peachy-6903604.txt'がインデックス化されました。ID: 5QStspIBHiy3rAkzORpQ
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/peachy-6907463.txt'がインデックス化されました。ID: 5gStspIBHiy3rAkzORpc
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/movie-enter-6908515.txt'がインデックス化されました。ID: 5wStspIBHiy3rAkzORpp
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/it_life-hack-6915190.txt'がインデックス化されました。ID: 6AStspIBHiy3rAkzORp4
ファイル'/Users/pci/Desktop/for_study/learn_ElasticSearch/news_search/news_data/topic-news-6907153.txt'がインデックス化されました。ID: 6QStspIBHiy3rAkzORqF
ファイル'/Users/pci/Desktop/for_study/learn_Ela

In [3]:
# 検索試し
res = es.search(
    index="news_articles",
    query={
            "match": {
                "published_at": "2012-08-29"
            }
        }
)

res.body

{'took': 80,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'news_articles',
    '_id': 'wgStspIBHiy3rAkzNxok',
    '_score': 1.0,
    '_source': {'url': 'http://news.livedoor.com/article/detail/6897614/',
     'published_at': '2012-08-29T10:00:00+0900',
     'title': '神を自負した男が、無限の野心の果てに追い求めたもの',
     'content': '8月4日からの先行上映を経て、いよいよ24日に全国公開となったリドリー・スコット監督最新作、人類最大の謎《人類の起源》に迫る3Dエピック・ミステリー超大作『プロメテウス』。25日（土）と26日（日）の2日間で20万人以上を動員し、ネットでは本編に散りばめられた謎の数々に、早くも謎解き合戦が過熱。その謎を解き明かす手掛かりを握る男からのメッセージ映像を独占入手した。\n\n\u3000『プロメテウス』では、ノオミ・ラパス演じる主人公のエリザベスが世界各地の古代遺跡から発見した共通のサインに導かれ、宇宙の果てへと旅立つ。その船となるプロメテウス号の開発や、アンドロイドのデヴィッドを含む17名の探査チームの選抜など、1兆ドルにも及ぶ巨額資金を提供するのが、ガイ・ピアース演じる巨大企業ウェイランド社の創設者ピーター・ウェイランドだ。\n\n\n\n\u3000劇場公開前にオープンした「PROJECT PROMETHEUS」というサイトでは、ウェイランド社の会社概要など『プロメテウス』の世界観に現実味を与える様々な設定が事細かに掲載されている他、本編では使用されていない多数の特別映像がアップされた。その一つである「PETER WEYLAND 2023 TED TALK」では、ピーター・ウェ